In [1]:
# Use server from examples/servers/streamable-http-stateless/
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_mcp_adapters.client import MultiServerMCPClient

load_dotenv()

shared_memory = MemorySaver()

In [11]:
# pyright: strict = false
# type: ignore
async def call_todo_agent(config: RunnableConfig, memory: MemorySaver) -> None:
    """Run a class todo agent using the react agent framework and a remote MCP server."""
    client = MultiServerMCPClient({
        "fastapi-mcp": {
            "url": "http://localhost:8000/mcp",
            "transport": "sse",
        }
    })
    system_prompt = (
        "You are a helpful and intelligent Todo Agent. "
        "Your primary tasks include adding, completing, listing, and deleting todos, along with other todo-related actions. "
        "You have access to specialized tools: 'todo tools' and 'todo collection tools' to assist with these tasks. "
        "Before performing any action, first check your memory to see if the required information is already available. "
        "Only use a tool if the necessary data is not found in memory. "
        "When using a tool, be thoughtful and deliberate—choose the most appropriate tool for the specific task at hand. "
        "Use the tool name explicitly in your response when invoking a tool."
    )

    remote_tools = await client.get_tools()
    graph = create_react_agent(
        "google_genai:gemini-2.0-flash",
        tools=remote_tools,
        checkpointer=memory,
        prompt=system_prompt,
    )

    # Create event stream
    events = graph.astream_events({
        "messages": [
            HumanMessage(
                content="yes go ahead ",
            ),
        ]
    }, config=config, version="v2")

    # Handle stream output
    async for event in events:
        if event["event"] == "on_chat_model_stream":
            if "chunk" in event["data"]:
                print(event["data"]["chunk"].content, end="", flush=True)

In [12]:

# Define thread config
thread_config = RunnableConfig(
    {"configurable": {
        "thread_id": 21,
    }}
)

await call_todo_agent(config=thread_config, memory=shared_memory)

Okay, I can help you plan your tomorrow. As a developer, here's a possible schedule with tasks grouped into categories:

**Work:**

*   **Code Review:** Review pull requests from team members (Morning - 1-2 hours).
*   **Bug Fixes:** Address high-priority bugs reported in the current sprint (Morning/Afternoon - 2-3 hours).
*   **Sprint Planning:** Participate in the sprint planning meeting to estimate and assign tasks for the next sprint (Afternoon - 1 hour).
*   **Documentation:** Update documentation for the feature you're currently developing (Afternoon - 1 hour).
*   **Learning/Research:** Dedicate time to learn a new technology or research a solution for a challenging problem (Evening - 1 hour).

**Personal:**

*   **Exercise:** Go for a run, hit the gym, or do a home workout (Morning - 30-45 minutes).
*   **Groceries:** Buy groceries for the week (Evening - 1 hour).
*   **Meal Prep:** Prepare lunch for the next day (Evening - 30 minutes).
*   **Read:** Read a book or article for 